In [1]:
import numpy as np
import pandas as pd
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict

In [2]:
#Train with 20x20 and random grid, test with full grid

#data = pd.read_csv("./classif/fullgrid/classifgridmon_callout.csv") #to predict
data = pd.read_csv("./classif/fullgrid/classifgridwk_callout.csv")
#data2 = pd.read_csv("./classif/20x20/classif20x20_callout.csv") #daily
#data3 = pd.read_csv("./classif/randomgrid/classif-randomgrid_callout.csv") #daily
#data = pd.read_csv("./classif/20x20/classif20x20frid_callout.csv")
#data = pd.read_csv("./classif/20x20/classif20x20monminute_callout.csv")
data2 = pd.read_csv("./classif/20x20/ml_v4/classif20x20week_callout.csv") #weekly, 5 labels
#data2 = pd.read_csv("./classif/20x20/ml_v3/classif20x20week_callout.csv") #weekly, 6 labels

data2.head()
#20x20 grid, monday(4/11) - callout

,callout0,callout1,callout2,callout3,callout4,callout5,callout6,callout7,callout8,callout9,...,callout163,callout164,callout165,callout166,callout167,home,high_dens,mid_dens,low_dens,work
0,0.004171,0.002020,0.001136,0.002224,0.001229,0.001006,0.003732,0.017355,0.045097,0.081238,...,0.051290,0.034896,0.029359,0.015874,0.010176,1,0,1,0,0
1,0.017197,0.067118,0.085808,0.151127,0.081362,0.078319,0.038332,0.016627,0.048560,0.141606,...,0.049536,0.040305,0.022394,0.022973,0.008775,1,0,1,0,0
2,0.030395,0.094464,0.118034,0.205434,0.128594,0.113804,0.051204,0.022694,0.076637,0.206301,...,0.053865,0.041385,0.020162,0.021785,0.009832,1,0,1,0,0
3,0.017531,0.035968,0.045582,0.088067,0.045359,0.038406,0.016841,0.028373,0.109677,0.182829,...,0.085003,0.068662,0.042476,0.029336,0.021122,1,0,1,0,0
4,0.011573,0.017074,0.023002,0.054133,0.015260,0.011425,0.005390,0.028782,0.119908,0.171088,...,0.100433,0.080173,0.052234,0.034012,0.027283,1,0,1,0,1


In [3]:
y = data2.iloc[:,168:] #h/w
#y = y.append(data3.iloc[:,26])

x = data2.iloc[:,:168] #sms, calls, internet
#x = x.append(data3.iloc[:,2:26])

#y = data.iloc[:,146] #h/w
#x = data.iloc[:,2:146] #sms, calls, internet
x_test = data.iloc[:,2:170]
#x_test

In [4]:
#y

In [5]:
#x

In [6]:
classifier = LabelPowerset(
    classifier = RandomForestClassifier(max_depth=2, random_state=0),require_dense=[False,True]
)
classifier.fit(x,y)
scores1 = cross_val_score(classifier, x, y, cv=5)
scores2 = cross_val_score(classifier, x, y, cv=10)

In [7]:
print("5 fold: %0.2f accuracy with a standard deviation of %0.2f" % (scores1.mean(), scores1.std()))
print("10 fold: %0.2f accuracy with a standard deviation of %0.2f" % (scores2.mean(), scores2.std()))

5 fold: 0.40 accuracy with a standard deviation of 0.08
10 fold: 0.39 accuracy with a standard deviation of 0.11


In [8]:
y_predt = classifier.predict(x_test) #predict labels for all the grid (100x100)

#model = KMeans(n_clusters=5)
#model = AffinityPropagation(random_state=5) - doesn't converge
#yhat=model.fit_predict(x_test)
y_predt.todense()

matrix([[0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1]])

In [10]:
predicted = pd.DataFrame(y_predt.todense(),columns=["home","high_dens", "mid_dens", "low_dens", "work"])
predicted['cellid'] = data.cellid
predicted.head()

,home,high_dens,mid_dens,low_dens,work,cellid
0,0,0,0,1,1,1
1,0,0,0,1,1,2
2,0,0,0,1,1,3
3,0,0,0,1,1,4
4,0,0,0,1,1,5


In [11]:
predicted.to_csv('./csv/pred_fullgrid_ml5_lp_milan.csv')